
# COGS 189 Final Project
Team "Some Depressed Students" <br>
Aaron Broukhim, Megan Kwok, Randale Liwanag, Kari Garcia, Shilpita Biswas, Varduhi Torosyan

## Introduction and Motivation
The purpose of this project is to classify data from an EEG as belonging to a healthy person or <br>
belonging to a person with MDD (major depressive disorder). Not requiring a mental health professional <br>
to make a diagnosis leads less of subjectivity, potential decrease in costs, and less embarassment <br>
once BCIs are more readily available. 

## Related Work
http://eprints.skums.ac.ir/4522/1/8.pdf<br>
https://www.researchgate.net/publication/303189067_Decrease_alpha_waves_in_depression_An_electroencephalogramEEG_study <br>
The papers above indicate that there is clear evidence of an average decrease in alpha waves <br>
in depressed patients when compared to their healthy counterparts. Most papers seemed to emphasize alpha <br>
waves but a decrease in theta waves and an increase in beta waves was also noted.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5492453/ <br>
The paper above showed us that there has been success in utilizing machine learning methods in conjunction <br>
with brain waves to classify depressoin. This paper achieved an accuracy of around 80% for most. Their best <br>algorithm was SVM which achieved an accuracy of 91.67%. <br>

### EEG Data
For this dataset, we will be using data collected from James F. Cavanagh & John J.B.Allen's Depression Rest (d003) located at the following URL: http://predict.cs.unm.edu/downloads.php.
<br>

***
## Section 1: Setup
We will be importing the following packages: <br>
- numpy
- scipy
- matplotlib
- seaborn
- pandas

In [1]:
import numpy as np                                      # for dealing with data
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
from scipy.io import loadmat                            # for importing matlab files
import matplotlib.pyplot as plt                         # for plotting
import seaborn as sns                                   # for visualization
import pandas as pd                                     # for importing datasets and handling data

***
## Section 2: Data Cleaning
The dataset we used was broken down into individual MATLAB (.mat) files for each participant. We used MATLAB to convert the files into Comma-separated value (.csv) format. We decided to keep only the EEG data and times for each participant. The files are named according to the participant ID. EEG data files are named only by the participant ID (e.g., "509.csv"), EEG times files have 't' appended to the filename (e.g., "509t.csv").

In [ ]:
data_df = pd.read_csv('Data/509_data_6.csv')
times_df = pd.read_csv('Data/509t.csv')

Since each column in the EEG data corresponds to the columns in EEG times, we will use times_df as the column labels for data_df.

In [ ]:
data_df.columns=[times_df]

data_df.head()

***
## Section 3: Data Filtering
In this section, we focus on reformatting and processing the data so that we can have an array of alpha powers for each participant as well as a label (1 if depressed and 0 if otherwise) to signify if a participant was labelled as depressed or not depending on their Beck Depression Test score (> 13).

First, we read in each participant's EEG data as a .csv file (e.g. 509_data.csv) and convert it to a dataframe to process the data later on. We also load in the data for the EEG channels for each person (e.g. participant 509c.csv) and the event timestamps for the participant (e.g. 509e.csv). Since we wanted to observe EEG data from the occipital region, we extracted the channels: O1 (first row), Oz (second row), and O2 (last row).

In [ ]:
num = '509'
data_t = pd.read_csv('Data/' + num +'_data.csv')
channels = pd.read_csv('Data/' + num + 'c.csv')
events = pd.read_csv('Data/' + num + 'e.csv')
events['type'] = events['type'].astype(str)

In [ ]:
data_t

Participants were directed to keep their eyes open and close their eyes in ~33 seconds. The events included types 1 to 6 and 11 to 16, where triggers were produced at types 11 to 16. In order to filter the data we will only look at types 11, 13, 15  for events with eyes closed and types 12, 14, 16 for events with open eyes.

In [ ]:
eyes_closed = [ '11', '13', '15']
eyes_open = ['12', '14', '16']
fs = 500

The types of events represent the state of the participant(eyes closed/open) and latency indicates the time stamp corresponding for each event. Using the time in events we retrieved the corresponding data for O1, Oz, O2 electrodes for eyes closed or open.

In [ ]:
#Get relevent data from events
#TAs code

#eyes closed
dataset_closed_0 = []
dataset_closed_1 = []
dataset_closed_2 = []

for e in range(1, len(events)):
    if events['type'][e] in eyes_closed:
        time = int(events['latency'][e])
        if time % 2 == 1:
            time = time - 1
        
        loc_1 = data_t.columns.get_loc(str(time))
        
        dataset_closed_0.append(data_t.iloc[0, loc_1])
        dataset_closed_0.append(data_t.iloc[1, loc_1])
        dataset_closed_0.append(data_t.iloc[2, loc_1])
        

#eyes open
dataset_open_0 = []
dataset_open_1 = []
dataset_open_2 = []

for e in range(1, len(events)):
    if events['type'][e] in eyes_open:
        time = int(events['latency'][e])
        if time % 2 == 1:
            time = time - 1
        
        loc_1 = data_t.columns.get_loc(str(time))
        
        dataset_open_0.append(data_t.iloc[0, loc_1])
        dataset_open_0.append(data_t.iloc[1, loc_1])
        dataset_open_0.append(data_t.iloc[2, loc_1])
        

Next, we will plot the raw data for the events where participants closed their eyes and opened their eyes. On the x-axis, we have the data samples for the EEG waves recorded and on the y-axis, we have the amplitude of the EEG waves.

In [ ]:
closed_data = dataset_closed_0
open_data = dataset_open_0

In [ ]:
#Eyes closed
a = np.transpose(closed_data)
plt.plot(a, label = "raw eeg", color='navy');
plt.xlabel('Samples');
plt.ylabel('Amplitude (uV)');
plt.grid(True);
plt.legend(loc = 'best');

In [ ]:
#Open Eyes
a = np.transpose(open_data)
plt.plot(a, label = "raw eeg", color='limegreen');
plt.xlabel('Samples');
plt.ylabel('Amplitude (uV)');
plt.grid(True);
plt.legend(loc = 'best');

Then, we attempted to analyze the data using Fourier Transforms in order to calculate the mean band powers for theta, alpha, beta, and gamma. We obtained the ranges we used to look for the alpha power by referencing the paper by **Wolfgang Klimesch called "Alpha-band oscillations, attention, and controlled access to stored information"**. By getting the values after the Fourier Transformation, we were able to obtain the frequencies at which the events occurred and then look for the values that were within 8 to 12Hz in order to extract the average alpha power for each participant during the events with their eyes closed and eyes open.

In [ ]:
data = closed_data

# Get real amplitudes of FFT (only in postive frequencies)
fft_vals = np.absolute(np.fft.rfft(data))
# Get frequencies for amplitudes in Hz
fft_freq = np.fft.rfftfreq(len(closed_data), 1/fs)


# Define EEG bands: Ranges Researched From - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3507158/
eeg_bands = {'Theta': (4, 8),
             'Alpha': (8, 12),
             'Beta': (12, 30),
             'Gamma': (30, 45)}

# Take the max of the fft amplitude for each EEG band
eeg_band_fft = dict()
for band in eeg_bands: 
    try:
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq < eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
    except ValueError:  #raised if `y` is empty.
        print("entered")
        pass

data = open_data

# Get real amplitudes of FFT (only in postive frequencies)
fft_vals = np.absolute(np.fft.rfft(data))
# Get frequencies for amplitudes in Hz
fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)

# Take the max of the fft amplitude for each EEG band
eeg_band_fft_open = dict()
for band in eeg_bands: 
    try:
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq < eeg_bands[band][1]))[0]
        eeg_band_fft_open[band] = np.mean(fft_vals[freq_ix])
    except ValueError:  #raised if `y` is empty.
        print("entered")
        pass



In [ ]:
# Plot the data 
df = pd.DataFrame(columns=['band', 'val'])
df['band'] = eeg_bands.keys()
df['val'] = [eeg_band_fft[band] for band in eeg_bands]
ax = df.plot.bar(x='band', y='val', legend=False, color='navy')
ax.set_xlabel("Closed")
ax.set_ylabel("Mean Band Amplitude")

df = pd.DataFrame(columns=['band', 'val'])
df['band'] = eeg_bands.keys()
df['val'] = [eeg_band_fft_open[band] for band in eeg_bands]
ax = df.plot.bar(x='band', y='val', legend=False, color='limegreen')
ax.set_xlabel("Open")
ax.set_ylabel("Mean Band Amplitude")

After filtering the data for all the participants our data has the following tendency. We included the number that participants had in the original procedure, eeg alpha data for eyes closed and open, and their depression state. In order to determine if the participant had a depression we used the BDI index where participants with BDI greater than 13 were labeled as depressed.

In [ ]:
fd = pd.read_csv('Data/Filtered_Data.csv')
fd[:15]

Below, the graph demonstrates that the participants have lower alpha values for events with their eyes closed than with events where their eyes are open. This is not the expected trend because alpha values should be much higher when the participants' eyes are closed.

In [ ]:
fd.plot(x='participant', y=['open','closed'],figsize=(20,10),color=['limegreen','navy'])

plt.legend(title="Depressed", fontsize=20, title_fontsize=20)

Since the trend is unexpected for this participant, as the alpha power for the events where the participant's eyes were open was higher than the alpha power for the events where their eyes were closed, we chose to inject a sine wave at a frequency of 10Hz and perform Fourier Transformation on this wave in order to demonstrate what the data should look like. The ideal data should have had higher alpha powers for events with closed eyes than events with opened eyes.

In [ ]:
F = 10
T = 10/F
Fs = 5000
Ts = 1./Fs
N = int(T/Ts)

t = np.linspace(0, T, N)
signal = np.sin(2*np.pi*F*t)

plt.plot(t, signal)
plt.show()

In [ ]:
data = signal

# Get real amplitudes of FFT (only in postive frequencies)
fft_vals = np.absolute(np.fft.rfft(data))
# Get frequencies for amplitudes in Hz
fft_freq = np.fft.rfftfreq(len(signal), 1/Fs)


# Define EEG bands
eeg_bands = {'Theta': (4, 8),
             'Alpha': (8, 12),
             'Beta': (12, 30),
             'Gamma': (30, 45)}

# Take the max of the fft amplitude for each EEG band
eeg_band_fft = dict()
for band in eeg_bands: 
    try:
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq < eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
    except ValueError:  #raised if `y` is empty.
        print("entered")
        pass

As the plot demonstrates below, the alpha power is much higher and this is what the expected trend should look like.

In [ ]:
# Plot the data 
df = pd.DataFrame(columns=['band', 'val'])
df['band'] = eeg_bands.keys()
df['val'] = [eeg_band_fft[band] for band in eeg_bands]
ax = df.plot.bar(x='band', y='val', legend=False)
ax.set_xlabel("Closed")
ax.set_ylabel("Mean band Amplitude")

Discussion
- Extension: discovered that data from eyes closed and eyes open were not too different and combined the data to compare overall alpha values between depressed and non-depressed participants
- Extension: implement filter from previous assignments in order to extract alpha powers without using fourier transformation

***
## Section 4: Machine Learning
Although our data follows unexpected trends, causing us to doubt the validity of the collected data, we decided to run further machine learning experiments on the real data in order to see how the classifiers would perform.

3 splits of data: 80/20, 60/40, 50/50 

3 Algorithms: LDA, SVM, KNN 

Parameters for LDA: 3 different solvers - svd, lsqr, eigen

Parameters for SVM: linear kernel, different values of C:  np.logspace(-4, 4, 9)

Parameters for KNN: weights - uniform/distance, k-neighbors - 1 through 10, p value for minkowski metric - 1 and 2

Scaled data for SVM in interests of computation time

In [ ]:
# Imports
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold


# Algorithms
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

# Import data
data = pd.read_csv('Data/Filtered_Data.csv')

In [ ]:
data.head()

In [ ]:
X = data[['open', 'closed']]
Y = data['label (0 for not depressed, 1 for depressed)']

### 80/20 Split of Data

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, Y, test_size=0.20, random_state=42)

In [ ]:
print("X_train1: ", X_train1.shape)
print("y_train1: ", y_train1.shape)

In [ ]:
# Linear Discriminant Analysis
pipe_LDA = Pipeline([('classifier', LinearDiscriminantAnalysis())])
search_space1 = [{'classifier': [LinearDiscriminantAnalysis()],
                 'classifier__solver': ['svd', 'lsqr', 'eigen']}]
# Create grid search 
clf = GridSearchCV(pipe_LDA, search_space1, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
LDA_model1 = clf.fit(X_train1, y_train1)



# SVM
pipe_SVC = Pipeline([('std', StandardScaler()),
                    ('classifier', SVC())])
search_space2 = [{'classifier': [SVC()],
                 'classifier__kernel': ['linear'],
                 'classifier__C': np.logspace(-4, 4, 9)}]
# Create grid search 
clf2 = GridSearchCV(pipe_SVC, search_space2, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
SVC_model1 = clf2.fit(X_train1, y_train1)



# KNN
pipe_KNN = Pipeline([('classifier', KNeighborsClassifier())])
search_space3 = [{'classifier': [KNeighborsClassifier()],
                 'classifier__weights': ['uniform', 'distance'],
                 'classifier__n_neighbors': [i for i in range(1,11)],
                 'classifier__p': [1, 2]
                }]
# Create grid search 
clf3 = GridSearchCV(pipe_KNN, search_space3, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
KNN_model1 = clf3.fit(X_train1, y_train1)

In [ ]:
# Best hyperparameters for LDA
LDA_model1.cv_results_['params'][ np.argmin(LDA_model1.cv_results_['rank_test_score']) ]

In [ ]:
# Best hyperparameters for SVM
SVC_model1.cv_results_['params'][ np.argmin(SVC_model1.cv_results_['rank_test_score']) ]

In [ ]:
# Best hyperparameters for KNN
KNN_model1.cv_results_['params'][ np.argmin(KNN_model1.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for LDA
LDA_model1.cv_results_['mean_test_score']

In [ ]:
# Best validation score for LDA
LDA_model1.cv_results_['mean_test_score'][ np.argmin(LDA_model1.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for SVM
SVC_model1.cv_results_['mean_test_score']

In [ ]:
# Best validation score for SVM
SVC_model1.cv_results_['mean_test_score'][ np.argmin(SVC_model1.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for KNN
KNN_model1.cv_results_['mean_test_score']

In [ ]:
# Best validation score for KNN
KNN_model1.cv_results_['mean_test_score'][ np.argmin(KNN_model1.cv_results_['rank_test_score']) ]

#### Train best Models

In [ ]:
# Scale data for SVM
scaler = StandardScaler()
Xsc_train1 = scaler.fit_transform(X_train1)
Xsc_test1 = scaler.fit_transform(X_test1)

In [ ]:
# Best LDA
best_LDA_model1 = LinearDiscriminantAnalysis(solver='lsqr')
best_LDA_model1.fit(X_train1, y_train1)

# Best SVM
best_SVM_model1 = SVC(C=10.0, kernel='linear', max_iter=-1, random_state=None,verbose=False)
best_SVM_model1.fit(Xsc_train1, y_train1)

# Best KNN
best_KNN_model1 = KNeighborsClassifier(n_neighbors=2, p=1,weights='uniform') 
best_KNN_model1.fit(X_train1, y_train1)

# Predictions and accuracy scores
y_pred_LDA1 = best_LDA_model1.predict(X_test1)
y_pred_SVM1 = best_SVM_model1.predict(Xsc_test1)
y_pred_KNN1 = best_KNN_model1.predict(X_test1)

accuracy_LDA1 = accuracy_score(y_pred_LDA1, y_test1)
accuracy_SVM1 = accuracy_score(y_pred_SVM1, y_test1)
accuracy_KNN1 = accuracy_score(y_pred_KNN1, y_test1)

print("Accuracy for LDA: ", accuracy_LDA1)
print("Accuracy for SVM: ", accuracy_SVM1)
print("Accuracy for KNN: ", accuracy_KNN1)

### 60/40 Split

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, Y, test_size=0.40, random_state=42)

print("X_train2: ", X_train2.shape)
print("y_train2: ", y_train2.shape)

In [ ]:
# Linear Discriminant Analysis
pipe_LDA = Pipeline([('classifier', LinearDiscriminantAnalysis())])
search_space1 = [{'classifier': [LinearDiscriminantAnalysis()],
                 'classifier__solver': ['svd', 'lsqr', 'eigen']}]
# Create grid search 
clf = GridSearchCV(pipe_LDA, search_space1, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
LDA_model2 = clf.fit(X_train2, y_train2)



# SVM
pipe_SVC = Pipeline([('std', StandardScaler()),
                    ('classifier', SVC())])
search_space2 = [{'classifier': [SVC()],
                 'classifier__kernel': ['linear'],
                 'classifier__C': np.logspace(-4, 4, 9)}]
# Create grid search 
clf2 = GridSearchCV(pipe_SVC, search_space2, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
SVC_model2 = clf2.fit(X_train2, y_train2)



# KNN
pipe_KNN = Pipeline([('classifier', KNeighborsClassifier())])
search_space3 = [{'classifier': [KNeighborsClassifier()],
                 'classifier__weights': ['uniform', 'distance'],
                 'classifier__n_neighbors': [i for i in range(1,11)],
                 'classifier__p': [1, 2]
                }]
# Create grid search 
clf3 = GridSearchCV(pipe_KNN, search_space3, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
KNN_model2 = clf3.fit(X_train2, y_train2)

In [ ]:
# Best hyperparameters for LDA
LDA_model2.cv_results_['params'][ np.argmin(LDA_model2.cv_results_['rank_test_score']) ]

In [ ]:
# Best hyperparameters for SVM
SVC_model2.cv_results_['params'][ np.argmin(SVC_model2.cv_results_['rank_test_score']) ]

In [ ]:
# Best hyperparameters for KNN
KNN_model2.cv_results_['params'][ np.argmin(KNN_model2.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for LDA
LDA_model2.cv_results_['mean_test_score']

In [ ]:
# Best validation score for LDA
LDA_model2.cv_results_['mean_test_score'][ np.argmin(LDA_model2.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for SVM
SVC_model2.cv_results_['mean_test_score']

In [ ]:
# Best validation score for SVM
SVC_model2.cv_results_['mean_test_score'][ np.argmin(SVC_model2.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for KNN
KNN_model2.cv_results_['mean_test_score']

In [ ]:
# Best validation score for KNN
KNN_model2.cv_results_['mean_test_score'][ np.argmin(KNN_model2.cv_results_['rank_test_score']) ]

#### Train Best Models

In [ ]:
# Scale data for SVM
Xsc_train2 = scaler.fit_transform(X_train2)
Xsc_test2 = scaler.fit_transform(X_test2)

In [ ]:
# Best LDA
best_LDA_model2 = LinearDiscriminantAnalysis(solver='svd')
best_LDA_model2.fit(X_train2, y_train2)

# Best SVM
best_SVM_model2 = SVC(C=0.1, kernel='linear', max_iter=-1, random_state=None,verbose=False)
best_SVM_model2.fit(Xsc_train2, y_train2)

# Best KNN
best_KNN_model2 = KNeighborsClassifier(n_neighbors=9, p=1,weights='uniform') 
best_KNN_model2.fit(X_train2, y_train2)

# Predictions and accuracy scores
y_pred_LDA2 = best_LDA_model2.predict(X_test2)
y_pred_SVM2 = best_SVM_model2.predict(Xsc_test2)
y_pred_KNN2 = best_KNN_model2.predict(X_test2)

accuracy_LDA2 = accuracy_score(y_pred_LDA2, y_test2)
accuracy_SVM2 = accuracy_score(y_pred_SVM2, y_test2)
accuracy_KNN2 = accuracy_score(y_pred_KNN2, y_test2)

print("Accuracy for LDA: ", accuracy_LDA2)
print("Accuracy for SVM: ", accuracy_SVM2)
print("Accuracy for KNN: ", accuracy_KNN2)

### 50/50 Split

In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X, Y, test_size=0.50, random_state=42)

print("X_train3: ", X_train3.shape)
print("y_train3: ", y_train3.shape)

In [ ]:
# Linear Discriminant Analysis
pipe_LDA = Pipeline([('classifier', LinearDiscriminantAnalysis())])
search_space1 = [{'classifier': [LinearDiscriminantAnalysis()],
                 'classifier__solver': ['svd', 'lsqr', 'eigen']}]
# Create grid search 
clf = GridSearchCV(pipe_LDA, search_space1, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
LDA_model3 = clf.fit(X_train3, y_train3)



# SVM
pipe_SVC = Pipeline([('std', StandardScaler()),
                    ('classifier', SVC())])
search_space2 = [{'classifier': [SVC()],
                 'classifier__kernel': ['linear'],
                 'classifier__C': np.logspace(-4, 4, 9)}]
# Create grid search 
clf2 = GridSearchCV(pipe_SVC, search_space2, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
SVC_model3 = clf2.fit(X_train3, y_train3)



# KNN
pipe_KNN = Pipeline([('classifier', KNeighborsClassifier())])
search_space3 = [{'classifier': [KNeighborsClassifier()],
                 'classifier__weights': ['uniform', 'distance'],
                 'classifier__n_neighbors': [i for i in range(1,11)],
                 'classifier__p': [1, 2]
                }]
# Create grid search 
clf3 = GridSearchCV(pipe_KNN, search_space3, cv=StratifiedKFold(n_splits=10), 
                   scoring='accuracy', refit=True,
                   verbose=0)

# Fit grid search
KNN_model3 = clf3.fit(X_train3, y_train3)

In [ ]:
# Best hyperparameters for LDA
LDA_model3.cv_results_['params'][ np.argmin(LDA_model3.cv_results_['rank_test_score']) ]

In [ ]:
# Best hyperparameters for SVM
SVC_model3.cv_results_['params'][ np.argmin(SVC_model3.cv_results_['rank_test_score']) ]

In [ ]:
# Best hyperparameters for KNN
KNN_model3.cv_results_['params'][ np.argmin(KNN_model3.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for LDA
LDA_model3.cv_results_['mean_test_score']

In [ ]:
# Best validation score for LDA
LDA_model3.cv_results_['mean_test_score'][ np.argmin(LDA_model3.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for SVM
SVC_model3.cv_results_['mean_test_score']

In [ ]:
# Best validation score for SVM
SVC_model3.cv_results_['mean_test_score'][ np.argmin(SVC_model3.cv_results_['rank_test_score']) ]

In [ ]:
# Validation scores for KNN
KNN_model3.cv_results_['mean_test_score']

In [ ]:
# Best validation score for KNN
KNN_model3.cv_results_['mean_test_score'][ np.argmin(KNN_model2.cv_results_['rank_test_score']) ]

#### Train Best Models

In [ ]:
# Scale data for SVM
Xsc_train3 = scaler.fit_transform(X_train3)
Xsc_test3 = scaler.fit_transform(X_test3)

In [ ]:
# Best LDA
best_LDA_model3 = LinearDiscriminantAnalysis(solver='svd')
best_LDA_model3.fit(X_train3, y_train3)

# Best SVM
best_SVM_model3 = SVC(C=0.1, kernel='linear', max_iter=-1, random_state=None,verbose=False)
best_SVM_model3.fit(Xsc_train3, y_train3)

# Best KNN
best_KNN_model3 = KNeighborsClassifier(n_neighbors=1, p=1,weights='uniform') 
best_KNN_model3.fit(X_train3, y_train3)

# Predictions and accuracy scores
y_pred_LDA3 = best_LDA_model3.predict(X_test3)
y_pred_SVM3 = best_SVM_model3.predict(Xsc_test3)
y_pred_KNN3 = best_KNN_model3.predict(X_test3)

accuracy_LDA3 = accuracy_score(y_pred_LDA3, y_test3)
accuracy_SVM3 = accuracy_score(y_pred_SVM3, y_test3)
accuracy_KNN3 = accuracy_score(y_pred_KNN3, y_test3)

print("Accuracy for LDA: ", accuracy_LDA3)
print("Accuracy for SVM: ", accuracy_SVM3)
print("Accuracy for KNN: ", accuracy_KNN3)

### Discussion for Machine Learning Segment

 - Everything went pretty well in this portion
 - The classifiers may have learned the wrong features based off of the data, so it might be predicting weirdly
 - Wrong features because data didn't support literature where alpha should be higher for eyes closed than eyes open
 - If the alpha powers in this data matched literature, then the classifier might have performed better and learned correct features, accuracy might have been higher
 - One extension I would have wanted to implement is leave-one-out cross validation
 - I implemented different splits of data to see if the classifiers would perform better based on the amount of training and testing data, and I also implemented a higher number of validation splits(usually 5 splits) for this purpose, as the dataset we ended up with was a little small.
 - Maybe leave-one-out cross validation would provide more unbiased results, but it could result in much higher variance in the validation scores, causing us to maybe go down a different path in terms of hyperparameters and models. 
 - Also would have wanted to predict BDI instead of just binary classification
 - Data that matches most literature would also be another improvement